# VacationPy

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf
from pathlib import Path

# Import API key
from api_keys import geoapify_key

d:\Users\phamt\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sawakin,19.1059,37.3321,31.89,61,0,3.06,SD,2023-09-29 15:52:29
1,1,songea,-10.6833,35.6500,18.64,82,40,3.09,TZ,2023-09-29 15:52:30
2,2,waitangi,-43.9535,-176.5597,8.79,79,96,2.68,NZ,2023-09-29 15:52:31
3,3,golhisar,37.1459,29.5088,17.79,65,0,0.27,TR,2023-09-29 15:52:31
4,4,blackmans bay,-43.0167,147.3167,11.52,72,88,3.64,AU,2023-09-29 15:52:32


In [3]:
cities_plot=gv.Dataset(city_data_df,kdims=["Lng","Lat"],vdims=["City","Country","Humidity"])
# Display the map
points=cities_plot.to(gv.Points,["Lng","Lat"],["City","Country","Humidity"]).opts(
    size=gv.dim("Humidity") * 0.1,  # Scale the size of points based on humidity
    tools=["hover"],  # Enable hover tools
    cmap='Category20',
    color='City',
    legend_position='right',
    alpha=0.7)

osm_map=gv.tile_sources.OSM
osm_map.opts(width=900, height=400) * points

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity)

In [4]:
# Finding ideal cities
ideal_cities_df=city_data_df.loc[(city_data_df['Max Temp']>21) & 
                                 (city_data_df['Max Temp']<27) &
                                 (city_data_df['Wind Speed']<4.5) &
                                 (city_data_df['Cloudiness']==0),:].reset_index()
ideal_cities_df

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,31,31,sainte-rose,16.3324,-61.6979,26.02,94,0,0.00,GP,2023-09-29 15:52:54
1,92,92,javanrud,34.7961,46.5172,23.81,22,0,1.84,IR,2023-09-29 15:53:33
2,159,159,brady,31.1352,-99.3351,26.20,53,0,3.60,US,2023-09-29 15:54:22
3,249,249,tindouf,27.6711,-8.1474,25.25,23,0,4.12,DZ,2023-09-29 15:55:24
4,271,271,paphos,34.7667,32.4167,26.88,61,0,3.60,CY,2023-09-29 15:55:39
5,275,275,bayeux,-7.1250,-34.9322,22.35,94,0,1.54,BR,2023-09-29 15:55:42
6,334,334,mersa matruh,31.3525,27.2453,22.11,94,0,1.54,EG,2023-09-29 15:56:24
7,368,368,black point,-34.6187,137.8972,24.83,35,0,2.51,AU,2023-09-29 15:56:46
8,390,390,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,2023-09-29 15:57:01
9,412,412,ha'il,27.5219,41.6907,26.01,18,0,4.12,SA,2023-09-29 15:57:18


In [5]:
# Create hotel_df includes city, country, coordinates, and humidity
hotel_df=ideal_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()
hotel_df

,City,Country,Lat,Lng,Humidity
0,sainte-rose,GP,16.3324,-61.6979,94
1,javanrud,IR,34.7961,46.5172,22
2,brady,US,31.1352,-99.3351,53
3,tindouf,DZ,27.6711,-8.1474,23
4,paphos,CY,34.7667,32.4167,61
5,bayeux,BR,-7.1250,-34.9322,94
6,mersa matruh,EG,31.3525,27.2453,94
7,black point,AU,-34.6187,137.8972,35
8,remire-montjoly,GF,4.9167,-52.2667,94
9,ha'il,SA,27.5219,41.6907,18


In [6]:
# Set base url
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
radius=10000
category="accommodation"
sort= "distance"
limit= 1

params={"radius":radius,
       "categories":category,
       "apiKey": geoapify_key,
       "sort": sort,
       "limit": limit}

# Hotel count
hotel_count=0
# Print a message to follow up the hotel search
print(f"Starting hotel search in {len(hotel_df)} cities")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get city coordinates
    lat,lng=row["Lat"],row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude
    params["lat"]=lat
    params["lon"]=lng
    
    # Make and API request using the params dictionaty
    name_address=requests.get(base_url,params=params).json()
    
    print(f"Searching in %s, %s..." % (row["City"],row["Country"]))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        print("Hotel found !!!")
        hotel_count+=1
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        pass
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    # Print address if hotel found
    if hotel_df.loc[index, 'Hotel Name']!="No hotel found":
        print("Hotel's address: %s." % (name_address["features"][0]["properties"]["address_line2"]))
    print("------------")
print("Searching completed. %s/%s found." % (hotel_count,len(hotel_df)))
hotel_df

Starting hotel search in 19 cities
Searching in sainte-rose, GP...
Hotel found !!!
sainte-rose - nearest hotel: Au son des grenouilles
Hotel's address: Chemin café, 97115 Sainte-Rose, France.
------------
Searching in javanrud, IR...
Hotel found !!!
javanrud - nearest hotel: زاولی
Hotel's address: دهستان شمشیر, شمالی کوه شاهو از روستای دیوازناو تا سریاس, Kermanshah Province, Iran.
------------
Searching in brady, US...
Hotel found !!!
brady - nearest hotel: Hilltop Motel
Hotel's address: 910 Frisco Avenue, Menard, TX 76859, United States of America.
------------
Searching in tindouf, DZ...
Hotel found !!!
tindouf - nearest hotel: محمد بوسبي
Hotel's address: شارع الطيب بولحروف, 37100 Tinduf, Algeria.
------------
Searching in paphos, CY...
Hotel found !!!
paphos - nearest hotel: Mayfair Hotel
Hotel's address: Pari, 8049 Paphos Municipality, Cyprus.
------------
Searching in bayeux, BR...
bayeux - nearest hotel: No hotel found
------------
Searching in mersa matruh, EG...
Hotel found !!!

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sainte-rose,GP,16.3324,-61.6979,94,Au son des grenouilles
1,javanrud,IR,34.7961,46.5172,22,زاولی
2,brady,US,31.1352,-99.3351,53,Hilltop Motel
3,tindouf,DZ,27.6711,-8.1474,23,محمد بوسبي
4,paphos,CY,34.7667,32.4167,61,Mayfair Hotel
5,bayeux,BR,-7.1250,-34.9322,94,No hotel found
6,mersa matruh,EG,31.3525,27.2453,94,صقور
7,black point,AU,-34.6187,137.8972,35,The Top Pub
8,remire-montjoly,GF,4.9167,-52.2667,94,Motel du Lac
9,ha'il,SA,27.5219,41.6907,18,فندق كلاسيك


In [7]:
hotels_plot=gv.Dataset(hotel_df,kdims=["Lng", "Lat"], vdims=["City", "Humidity", "Country"])

# Display the map
points=hotels_plot.to(gv.Points,["Lng","Lat"],[ "Humidity","City","Country","Hotel Name"]).opts(
    size=gv.dim("Humidity") * 0.2,  # Scale the size of points based on humidity
    tools=["hover"],  # Enable hover tools
    cmap='Category20', # Choose a color map for city names
    color='City', # Assign unique colors based on city names
    legend_position='right', # Place legend to the right of the plot
    alpha=0.9) # Set transparency

osm_map=gv.tile_sources.OSM

osm_map.opts(width=900, height=400) * points

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)